# Assignment 7

### Part A

In [22]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [23]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [24]:
def expected_return(P, rf, mkt_rp):
    newDF = pd.DataFrame(P)
    newDF["ER"] = rf + newDF["Beta"]*mkt_rp
    total = newDF["Amount Invested"].sum()
    newDF["pct"] = newDF["Amount Invested"]/total
    newDF["ER2"] = newDF["ER"]*newDF["pct"]

    # Write your code here
    return newDF["ER2"].sum()

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [26]:
def volatility(P, cor):
    newDF = pd.DataFrame(P)
    total = newDF["Amount Invested"].sum()
    newDF["pct"] = newDF["Amount Invested"]/total
    newDF["Volpct^2"] = (newDF["pct"]*newDF["Volatility"])**2
    newDF["Volpct^2"].sum() + 2*cor*newDF["pct"].product()*newDF["Volatility"].product()
    
    
    # Write your code here
    return (newDF["Volpct^2"].sum() + 2*cor*newDF["pct"].product()*newDF["Volatility"].product())**0.5

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [28]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    x = vol/mkt_vol
    # Write your code here
    return rf + x*mkt_rp


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [40]:
def CAPM_beta(return_path, factors_path):
    df = pd.read_csv(factors_path)
    df["Mkt-RF"]/=100
    df["RF"]/=100
    df2 = pd.read_csv(return_path)
    df2["date"] = df2["year"]*100 + df2["month"]
    
    
    merged = pd.merge(df2, df, on = "date", how="left")
    merged["R_i - Rf"] = merged["RET"] - merged["RF"]
    merged["R_mkt - Rf"] = merged["Mkt-RF"] #- merged["RF"]
    merged.drop(columns=["year", "month", "date", "Mkt-RF", "RF"], inplace=True)
    
    output = merged.groupby("CUSIP").agg(
        {
            "RET": "mean",
            "R_mkt - Rf": "var"
        }).reset_index()
    
    def calculate_covariance(x):
        return x['R_i - Rf'].cov(x['R_mkt - Rf'])
    output["cov"] = merged.groupby("CUSIP").apply(calculate_covariance).reset_index(drop = True)
    output["EXP_RETURN"] = output["RET"]
    output["BETA"] = output["cov"]/output["R_mkt - Rf"]
    
    
    output.drop(columns=["R_mkt - Rf", "cov", "RET"], inplace=True)
    
    
    
    
    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [42]:
def slope(data):
    slope, intercept = np.polyfit(data['BETA'], data['EXP_RETURN'], deg=1)
    # This should return 0.007981588120535642... with the provided data.
    return intercept